# Crush Rig - XGBoost
Written by Matt MacDonald for CIGITI at the Hospital for Sick Children Toronto
***

All tools to manipulate data will be obtained from the crush_plot.py file. The objective of this notebook is to predict the histological targets from the force/position crush data using xgboost.

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from crush_read import *

The crush data must be collected using the crush rig and crush.py and stored in the expected folder structure at the root directory indicated by PATH.

In [ ]:
# PATH = Path('')
# Default in crush_plot.py
PATH

Load all data and modify as needed.

In [ ]:
study = study_outline(PATH)
targets = study_targets(PATH)
crushes = study_data(study)
# crushes = split(crushes) # exclude multi-crushes
crushes = modify(crushes)
crushes = calculate(crushes)

Prepare data for xgboost.

In [ ]:
X, y, legend = prep(crushes, targets)
print('Reference for categorical features:')
legend

Drop Protocol from the features since only Stop is being assessed.

In [ ]:
X = X.drop('Protocol (cat)', axis=1)

In [ ]:
X.shape

Generate matrix of correlations to aid understanding.

In [ ]:
W = pd.concat([X, y], axis=1)
W_corr = W.corr(method='spearman')
sns.heatmap(W_corr, cmap='RdBu')

In [ ]:
W.describe()

In [ ]:
target_names = ['Damage Score', 'Percent Delta', 'P Score']
x_name = 'Target Stress (MPa)'
for y_name in target_names:
    plt.figure()
    plt.scatter(x=W[x_name], y=W[y_name])
    plt.xlabel(x_name)
    plt.ylabel(y_name)

Build xgboost model.

In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X.values, y.values, test_size=0.2, random_state=42)

In [ ]:
X_train.shape

In [ ]:
clf = XGBClassifier()
clf.fit(X_train, y_train)

In [ ]:
y_pred = clf.predict(X_test)
y_corr = y_pred == y_test

In [ ]:
y_pred_train = clf.predict(X_train)
y_corr_train = y_pred_train == y_train

In [ ]:
print(f"test acc = {sum(y_corr) / len(y_corr)}")
print(f"train acc = {sum(y_corr_train) / len(y_corr_train)}")

In [ ]:
from xgboost import plot_tree
plot_tree(clf, rankdir='LR', num_trees=3)

In [ ]:
# feature numbers legend
for i, feat in enumerate(X.columns):
    print(f"f{i} = {feat}")

In [ ]:
X.corr()

In [ ]:
df = pd.concat([X, y], axis=1)
df.corr()

In [ ]:
for prot in [0, 1]:
    prot_str = f"Protocol[{prot}]"
    avg = df.loc[df['Protocol'] == prot, 'Damage Score'].mean()
    print(f"{legend[prot_str]} average damage = {avg}")
          